<a href="https://colab.research.google.com/github/Satyam204/ML-Projects-Learning-/blob/main/Disease_prediction_using_symptoms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing the dependencies
import re
import pandas as pd
#import pyttsx3
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier,_tree
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import tree
import csv
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

**Loading the Datasets**

In [ ]:
training = pd.read_csv('/content/Training.csv')     #loading our training dataset
testing= pd.read_csv('/content/Testing.csv')         #loading our testing dataset
cols= training.columns
cols= cols[:-1]
x = training[cols]                                  #storing all symptoms features in x
y = training['prognosis']                           #storing target feature in y
y1= y

print(y.unique())

['Fungal infection' 'Allergy' 'GERD' 'Chronic cholestasis' 'Drug Reaction'
 'Peptic ulcer diseae' 'AIDS' 'Diabetes ' 'Gastroenteritis'
 'Bronchial Asthma' 'Hypertension ' 'Migraine' 'Cervical spondylosis'
 'Paralysis (brain hemorrhage)' 'Jaundice' 'Malaria' 'Chicken pox'
 'Dengue' 'Typhoid' 'hepatitis A' 'Hepatitis B' 'Hepatitis C'
 'Hepatitis D' 'Hepatitis E' 'Alcoholic hepatitis' 'Tuberculosis'
 'Common Cold' 'Pneumonia' 'Dimorphic hemmorhoids(piles)' 'Heart attack'
 'Varicose veins' 'Hypothyroidism' 'Hyperthyroidism' 'Hypoglycemia'
 'Osteoarthristis' 'Arthritis' '(vertigo) Paroymsal  Positional Vertigo'
 'Acne' 'Urinary tract infection' 'Psoriasis' 'Impetigo']


**Data Preprocessing ans Train-Test split**

In [ ]:
#The purpose of grouping the dataset by this column is likely to aggregate the
#data for each unique prognosis, so that we can get a sense of the maximum
#values for each feature (column) in the dataset for each prognosis.
reduced_data = training.groupby(training['prognosis']).max()

#Label encoding string type categorical data of training dataset into numerical data
#mapping strings to numbers
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)


#Splitting the training dataset into testing and train variables
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
testx    = testing[cols]
testy    = testing['prognosis']

le = preprocessing.LabelEncoder()
le.fit(testy)
testy    = le.transform(testy)

Accuracy and K-fold cross validation Scores of Our Model

In [ ]:
#Storing Decision Tree classifier of scikit library of python in clf1 and fitting
#our dataset in tree classification model

clf1  = DecisionTreeClassifier()
clf = clf1.fit(x_train,y_train)
#evaluates its performance on the training set
print(clf.score(x_train,y_train))
print("Accuracy result========")

#also calculates the cross-validation score using cross_val_score
#with 3-fold cross-validation on the test set
scores = cross_val_score(clf, x_test, y_test, cv=3)
print(scores)
print(scores.mean())

clf2=LogisticRegression()
clf22=clf2.fit(x_train,y_train)
print(clf22.score(x_train,y_train))
print("Accuracy result========")
scores2 = cross_val_score(clf22, x_test, y_test, cv=3)
print(scores2)
print(scores2.mean())

1.0
Accuracy result========
[0.96678967 0.97597043 0.97966728]
0.9741424586149744
1.0
Accuracy result========
[1. 1. 1.]
1.0


In [ ]:
#Graphical representation of our Decision Tree Classifier
fig = plt.figure(figsize=(125,100))
classes=training['prognosis']
features = training[training.columns[:-1]]
feature_names = list(features.columns)  # Get a list of feature names from the feature DataFrame
_ = tree.plot_tree(clf, feature_names=feature_names, class_names=classes, filled=True)

**Chatbot Working**

In [ ]:
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
features = cols


df1 = pd.read_csv('/content/Symptom-severity.csv')
df2 = pd.read_csv('/content/symptom_Description.csv')
df3 = pd.read_csv('/content/symptom_precaution.csv')
df4 = pd.read_csv('/content/symptom_medication.csv')
severityDictionary=df1.set_index('Symptom').T.to_dict('int')
description_list = df2.set_index('Disease').T.to_dict('list')
precautionDictionary=df3.set_index('Disease').T.to_dict('list')
medicationDictionary=df4.set_index('Disease').T.to_dict('list')

symptoms_dict = {}

for index, symptom in enumerate(x):
       symptoms_dict[symptom] = index

#The function calc_condition(exp, days) takes a list of symptoms and the
#number of days the patient has been experiencing them, calculates the
#total severity of the symptoms, and prints a message based on the severity of the symptoms.
def calc_condition(exp,days):
    sum=0
    for item in exp:
         item = item.replace(' ','')
         sum=sum+severityDictionary['weight'][item]
    if((sum*days)/(len(exp)+1)>13):
        print("You should take the consultation from doctor. ")
    else:
        print("It might not be that bad but you should take precautions.")

#The function getDescription() loads the description
#of diseases from the CSV file and stores them in a dictionary.
def getDescription():
    global description_list
    with open('/content/symptom_Description.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _description={row[0]:row[1]}
            description_list.update(_description)



#The function getSeverityDict() loads the symptom severity
#measures from the CSV file and stores them in a dictionary.
def getSeverityDict():
    global severityDictionary
    with open('/content/Symptom-severity.csv') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        try:
            for row in csv_reader:
                _diction={row[0]:int(row[1])}
                severityDictionary.update(_diction)
        except:
            pass

#The function getprecautionDict() loads the precaution
#measures for each disease from the CSV file and stores them in a dictionary.
def getprecautionDict():
    global precautionDictionary
    with open('/content/symptom_precaution.csv') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _prec={row[0]:[row[1],row[2],row[3],row[4]]}
            precautionDictionary.update(_prec)

#The function getInfo() takes user input for their name and greets them.
def getInfo():
    print("-----------------------------------HealthCare ChatBot-----------------------------------")
    print("\nYour Name? \t\t\t\t",end="->")
    name=input("")
    print("Hello, ",name)

#The function check_pattern(dis_list, inp) checks the input against
#the list of diseases and returns the matching diseases.
def check_pattern(dis_list,inp):
    pred_list=[]
    inp=inp.replace(' ','_')
    patt = f"{inp}"
    regexp = re.compile(patt)
    pred_list=[item for item in dis_list if regexp.search(item)]
    if(len(pred_list)>0):
        return 1,pred_list
    else:
        return 0,[]

#The function sec_predict(symptoms_exp) uses the Decision Tree
#Classification to predict the possible disease based on the input symptoms.
def sec_predict(symptoms_exp):
    df = pd.read_csv('/content/Training.csv')
    X = df.iloc[:, :-1]
    y = df['prognosis']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)
    rf_clf = DecisionTreeClassifier()
    rf_clf.fit(X_train, y_train)

    symptoms_dict = {symptom: index for index, symptom in enumerate(X)}
    input_vector = np.zeros(len(symptoms_dict))
    for item in symptoms_exp:
      input_vector[[symptoms_dict[item]]] = 1

    return rf_clf.predict([input_vector])

#The function print_disease(node) prints the predicted disease.
def print_disease(node):
    node = node[0]
    val  = node.nonzero()
    disease = le.inverse_transform(val[0])
    return list(map(lambda x:x.strip(),list(disease)))


#The function tree_to_code(tree, feature_names) recursively traverses the
#decision tree and returns the predicted disease based on the input symptoms.
#It also takes input from the user for the symptoms they are experiencing and
#the number of days they have been experiencing them.
def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    chk_dis=",".join(feature_names).split(",")
    symptoms_present = []

    while True:

        print("\nEnter the symptom you are experiencing  \t\t",end="->")
        disease_input = input("")
        conf,cnf_dis=check_pattern(chk_dis,disease_input)
        if conf==1:
            print("searches related to input: ")
            for num,it in enumerate(cnf_dis):
                print(num,")",it)
            if num!=0:
                print(f"Select the one you meant (0 - {num}):  ", end="")
                conf_inp = int(input(""))
            else:
                conf_inp=0

            disease_input=cnf_dis[conf_inp]
            break
            # print("Did you mean: ",cnf_dis,"?(yes/no) :",end="")
            # conf_inp = input("")
            # if(conf_inp=="yes"):
            #     break
        else:
            print("Enter valid symptom.")

    while True:
        try:
            num_days=int(input("Okay. From how many days ? : "))
            break
        except:
            print("Enter valid input.")
    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]

            if name == disease_input:
                val = 1
            else:
                val = 0
            if  val <= threshold:
                recurse(tree_.children_left[node], depth + 1)
            else:
                symptoms_present.append(name)
                recurse(tree_.children_right[node], depth + 1)
        else:
            present_disease = print_disease(tree_.value[node])
            # print( "You may have " +  present_disease )
            red_cols = reduced_data.columns
            symptoms_given = red_cols[reduced_data.loc[present_disease].values[0].nonzero()]
            # dis_list=list(symptoms_present)
            # if len(dis_list)!=0:
            #     print("symptoms present  " + str(list(symptoms_present)))
            # print("symptoms given "  +  str(list(symptoms_given)) )
            print("Are you experiencing any ")
            symptoms_exp=[]
            for syms in list(symptoms_given):
                inp=""
                print(syms,"? : ",end='')
                while True:
                    inp=input("")
                    if(inp=="yes" or inp=="no"):
                        break
                    else:
                        print("provide proper answers i.e. (yes/no) : ",end="")
                if(inp=="yes"):
                    symptoms_exp.append(syms)

            second_prediction=sec_predict(symptoms_exp)
            # print(second_prediction)
            calc_condition(symptoms_exp,num_days)
            if(present_disease[0]==second_prediction[0]):
                print("You may have ", present_disease[0])
                print(description_list[present_disease[0]])

                # readn(f"You may have {present_disease[0]}")
                # readn(f"{description_list[present_disease[0]]}")

            else:
                print("You may have ", present_disease[0], "or ", second_prediction[0])
                print(description_list[present_disease[0]])
                print(description_list[second_prediction[0]])

            # print(description_list[present_disease[0]])
            precution_list=precautionDictionary[present_disease[0]]
            symptom_list=medicationDictionary[present_disease[0]]
            print("Take following measures : ")
            for  i,j in enumerate(precution_list):
                print(i+1,")",j)
            print("Here are some medications which you can consider to use : ")
            for  i,j in enumerate(symptom_list):
                #if(j!=null):
                print(i+1,")",j)
            # confidence_level = (1.0*len(symptoms_present))/len(symptoms_given)
            # print("confidence level is " + str(confidence_level))

    recurse(0, 1)
getSeverityDict()
getDescription()
getprecautionDict()
getInfo()
tree_to_code(clf,cols)
print("----------------------------------------------------------------------------------------")

<ipython-input-24-90c2ef1ea318>:10: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  severityDictionary=df1.set_index('Symptom').T.to_dict('int')
<ipython-input-24-90c2ef1ea318>:10: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  severityDictionary=df1.set_index('Symptom').T.to_dict('int')


-----------------------------------HealthCare ChatBot-----------------------------------

Your Name? 				->ne
Hello,  ne

Enter the symptom you are experiencing  		->swelling
searches related to input: 
0 ) swelling_of_stomach
1 ) swelling_joints
Select the one you meant (0 - 1):  0
Okay. From how many days ? : 2
Are you experiencing any 
muscle_weakness ? : no
stiff_neck ? : no
swelling_joints ? : no
movement_stiffness ? : no
painful_walking ? : no
It might not be that bad but you should take precautions.
You may have  Arthritis or  Impetigo
Arthritis is the swelling and tenderness of one or more of your joints. The main symptoms of arthritis are joint pain and stiffness, which typically worsen with age. The most common types of arthritis are osteoarthritis and rheumatoid arthritis.
Impetigo (im-puh-TIE-go) is a common and highly contagious skin infection that mainly affects infants and children. Impetigo usually appears as red sores on the face, especially around a child's nose and m

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
